## Data Understanding

In [9]:
import pandas as pd
import numpy as np
import gc
import pickle
import sys
import warnings
sys.path.insert(1, "../scripts")
warnings.filterwarnings('ignore')

In [16]:
from data_cleaning import DataCleaner
from data_summary import DataSummarizer
cleaner = DataCleaner()
summary = DataSummarizer()

In [17]:
#read the raw data
raw_df = pd.read_pickle('../data/telecom_xdr.pkl')
#drop any duplicate rows
raw_df_droped_dup = raw_df.drop_duplicates()
#associeted column descriptions
colum_descript = pd.read_excel('../resources/Field Descriptions.xlsx')

In [18]:
# column names missmatch b/n data and description
[x for x in colum_descript['Fields'].tolist() if x not in raw_df.columns.tolist()]

['bearer id',
 'Dur. (s)',
 'YouTube DL (Bytes)',
 'YouTube UL (Bytes)',
 'Other DL',
 'Other UL']

In [19]:
[x for x in raw_df.columns.tolist() if x not in colum_descript['Fields'].tolist()]

['Bearer Id',
 'Youtube DL (Bytes)',
 'Youtube UL (Bytes)',
 'Other DL (Bytes)',
 'Other UL (Bytes)']

In [20]:
#Match columns in the description df to the data df
raw_df_renamed = raw_df.rename(columns={'Bearer Id':'bearer id' , 'Youtube DL (Bytes)':'YouTube DL (Bytes)', 'Youtube UL (Bytes)':'YouTube UL (Bytes)', 'Other DL (Bytes)':'Other DL', 'Other UL (Bytes)':'Other UL'})
del raw_df
gc.collect()

1311

## Data Pre-Processing
Data preprocessing is an integral step in Machine Learning as the quality of data and the useful information that can be derived from it directly affects the ability of our model to learn; therefore, it is extremely important that we preprocess our data before feeding it into our model.
#### Handling Null Values

In [21]:
#Original data summary
raw_df_renamed.info()
raw_df_renamed.head()
print(summary.percent_missing(raw_df_renamed,))
summary.summ_columns(raw_df_renamed)

The dataset contains 12.5 % missing values.
None


,variables,missing_count,missing_percent_(%),data_type,unique_values
0,bearer id,991,1.0,float64,134709
1,Start,1,0.0,object,9998
2,Start ms,1,0.0,float64,1001
3,End,1,0.0,object,6404
4,End ms,1,0.0,float64,1001
5,Dur. (ms),1,0.0,float64,89526
6,IMSI,570,0.0,float64,107266
7,MSISDN/Number,1066,1.0,float64,106857
8,IMEI,572,0.0,float64,107271
9,Last Location Name,1153,1.0,object,45548


#### Missing values & Data-Types

In [7]:
# Looking for nulls
print(raw_df_renamed.isnull().any())
# Inspecting type
print(raw_df_renamed.dtypes)

bearer id                                    True
Start                                        True
Start ms                                     True
End                                          True
End ms                                       True
Dur. (ms)                                    True
IMSI                                         True
MSISDN/Number                                True
IMEI                                         True
Last Location Name                           True
Avg RTT DL (ms)                              True
Avg RTT UL (ms)                              True
Avg Bearer TP DL (kbps)                      True
Avg Bearer TP UL (kbps)                      True
TCP DL Retrans. Vol (Bytes)                  True
TCP UL Retrans. Vol (Bytes)                  True
DL TP < 50 Kbps (%)                          True
50 Kbps < DL TP < 250 Kbps (%)               True
250 Kbps < DL TP < 1 Mbps (%)                True
DL TP > 1 Mbps (%)                           True


In [8]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.prod(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

def columns_missing_most_values(df, decript, percentage):
    colum_wise = df.isnull().sum() * 100 /df.shape[0]
    col_names = colum_wise[colum_wise > percentage].index.tolist() 
    print(" {} columns are missing moethan {}% of their values\nThey are:\n".format(len(col_names), percentage))
    with pd.option_context('expand_frame_repr', False):
        droped = decript.loc[decript['Fields'].isin(col_names)]
        droped['% Missing'] = [colum_wise.get(col_nm) for col_nm in col_names]
        print (droped)
    return col_names

percent_missing(raw_df_renamed)
# drop columns with more than 30% missing values
to_drop = columns_missing_most_values(raw_df_renamed, colum_descript, 30)
#I choose to fill the missing values than drop the columns
#But 'Dur. (ms)' & 'Dur. (ms).1' are the same in milli and micro seconds, let's drop the micro seconds column
raw_df_droped = raw_df_renamed.drop('Dur. (ms).1', axis=1)
del raw_df_renamed
gc.collect()


The dataset contains 12.5 % missing values.
 10 columns are missing moethan 30% of their values
They are:

                                      Fields                                        Description  % Missing
15               TCP DL Retrans. Vol (Bytes)  TCP volume of Downlink packets detected as ret...  58.763608
16               TCP UL Retrans. Vol (Bytes)  TCP volume of Uplink packets detected as retra...  64.432237
25                           HTTP DL (Bytes)  HTTP data volume (in Bytes) received by the MS...  54.315638
26                           HTTP UL (Bytes)  HTTP data volume (in Bytes) sent by the MS dur...  54.539636
32           Nb of sec with 125000B < Vol DL              Number of seconds with IP Volume DL >  65.024900
33     Nb of sec with 1250B < Vol UL < 6250B      Number of seconds with IP Volume UL between …  61.928920
34  Nb of sec with 31250B < Vol DL < 125000B      Number of seconds with IP Volume DL between …  62.390251
35            Nb of sec with 37500B <

/tmp/ipykernel_9354/3932666258.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  droped['% Missing'] = [colum_wise.get(col_nm) for col_nm in col_names]


0

In [9]:
len(raw_df_droped.columns)

54

In [10]:
def columns_missing_values(df, decript):
    colum_wise = df.isnull().sum() * 100 /df.shape[0]
    missing_col_names = colum_wise[colum_wise > 0].index.tolist()
    complete_col_names = [cn for cn in df.columns if cn not in missing_col_names]
    print(" {} columns are missing atleast 1 value, while {} have no missing values.\nThe columns missing values are:\n".format(len(missing_col_names), len(complete_col_names)))
    with pd.option_context('expand_frame_repr', False):
        with_missing = decript.loc[decript['Fields'].isin(missing_col_names)]
        with_missing['% Missing'] = [colum_wise.get(col_nm) for col_nm in missing_col_names]
        print (with_missing.sort_values(by=['% Missing']))
        print("\n The columns with full data are:\n")
        complete = decript.loc[decript['Fields'].isin(complete_col_names)]
        complete['% Missing'] = [colum_wise.get(col_nm) for col_nm in complete_col_names]
        print (complete)
    return missing_col_names, complete_col_names
# inspect the rest of the missimng values(their description)
to_fill, complet = columns_missing_values(raw_df_droped, colum_descript)

 40 columns are missing atleast 1 value, while 14 have no missing values.
The columns missing values are:

                                      Fields                                        Description  % Missing
55                          Total UL (Bytes)  Data volume (in Bytes) sent by the MS during t...   0.000667
1                                  Dur. (ms)                  Total Duration of the xDR (in ms)   0.000667
2                                      Start      Start time of the xDR (first frame timestamp)   0.000667
3                                   Start ms  Milliseconds offset of start time for the xDR ...   0.000667
4                                        End         End time of the xDR (last frame timestamp)   0.000667
5                                     End ms  Milliseconds offset of end time of the xDR (la...   0.000667
54                          Total DL (Bytes)  Data volume (in Bytes) received by the MS duri...   0.000667
14                   Avg Bearer TP UL

/tmp/ipykernel_9354/2102104408.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_missing['% Missing'] = [colum_wise.get(col_nm) for col_nm in missing_col_names]
/tmp/ipykernel_9354/2102104408.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete['% Missing'] = [colum_wise.get(col_nm) for col_nm in complete_col_names]


In [11]:
#print sorted no_of_unique values per column 
def columns_uniqueness(df, decript):
    col_wise_unique = df.nunique(dropna=True)
    col_names = col_wise_unique.index.tolist()
    uniques = decript.loc[decript['Fields'].isin(col_names)]
    uniques['uniques'] = [col_wise_unique.get(col_nm) for col_nm in col_names]
    with pd.option_context('expand_frame_repr', False):
        print (uniques.sort_values(by=['uniques']))

def print_sorted_no_uniques(df, decript):
    #for the columsn with the missing values
    print("No of unique values in the columns missing values")
    df_missing_values = df.drop(complet, axis=1)
    columns_uniqueness(df_missing_values, decript)

    #for the columns without any missing values
    print("\nNo of unique values in the columns without any missing values")
    df_complete_values = df.drop(to_fill, axis=1)
    columns_uniqueness(df_complete_values, decript)

print_sorted_no_uniques(raw_df_droped, colum_descript)

No of unique values in the columns missing values
                                      Fields                                        Description  uniques
24                      UL TP > 300 Kbps (%)  Duration ratio when Bearer Uplink Throughput > ….       63
23            50 Kbps < UL TP < 300 Kbps (%)  Duration ratio when Bearer Uplink Throughput r...       68
19             250 Kbps < DL TP < 1 Mbps (%)  Duration ratio when Bearer Downlink Throughput...       73
18            50 Kbps < DL TP < 250 Kbps (%)  Duration ratio when Bearer Downlink Throughput...       84
22             10 Kbps < UL TP < 50 Kbps (%)  Duration ratio when Bearer Uplink Throughput r...       85
20                        DL TP > 1 Mbps (%)  Duration ratio when Bearer Downlink Throughput...       85
21                       UL TP < 10 Kbps (%)  Duration ratio when Bearer Uplink Throughput < ….       98
17                       DL TP < 50 Kbps (%)  Duration ratio when Bearer Downlink Throughput...      100
30   

/tmp/ipykernel_9354/1916892460.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniques['uniques'] = [col_wise_unique.get(col_nm) for col_nm in col_names]
/tmp/ipykernel_9354/1916892460.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uniques['uniques'] = [col_wise_unique.get(col_nm) for col_nm in col_names]


##### !! The is no single unique column to be used as key
##### Hence, we will use the index as unique id for the rows 

In [12]:
raw_df_droped['index'] = raw_df_droped.index

##### The Columns can be grouped in to five different types:
* **Categorical**:  categorical properties
* **Time Series**:  system time stamps of events
* **Numerica speed ratios**: Performanc edistribution over speed ranges in %
* **Numerical speed**:  communication speed measured in Kbps or roundtrip time in milli-sec
* **Numerical time-gap**:   Time-gap measures in milli-sec
* **Numerical data-volume**:    Data volume measures in Bytes

In [13]:
categorical_columns = ['Handset Type', 'Handset Manufacturer', 'Last Location Name', 'IMEI', 'MSISDN/Number', 'IMSI', 'bearer id']
time_series_columns = ['Start', 'End', 'Start ms', 'End ms']
numeric_colmns_speed_ratios = ['DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)', '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)', 
    'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)', '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)']
numeric_colmns_speed = ['Avg RTT DL (ms)','Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)']
numeric_colmns_time = ['Dur. (ms)', 'Activity Duration DL (ms)','Activity Duration UL (ms)', 'Nb of sec with 125000B < Vol DL',
    'Nb of sec with 1250B < Vol UL < 6250B', 'Nb of sec with 31250B < Vol DL < 125000B', 'Nb of sec with 37500B < Vol UL', 
    'Nb of sec with 6250B < Vol DL < 31250B', 'Nb of sec with 6250B < Vol UL < 37500B', 'Nb of sec with Vol DL < 6250B',
    'Nb of sec with Vol UL < 1250B']
numeric_colmns_data_volume = ['Social Media DL (Bytes)', 'Social Media UL (Bytes)', 'Google DL (Bytes)', 'Google UL (Bytes)', 'Email DL (Bytes)',
    'Email UL (Bytes)', 'YouTube DL (Bytes)', 'YouTube UL (Bytes)', 'Netflix DL (Bytes)', 'Netflix UL (Bytes)', 'Gaming DL (Bytes)',
    'Gaming UL (Bytes)', 'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Other DL', 'Other UL', 'Total UL (Bytes)', 'Total DL (Bytes)', 
    'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)']

#### Fill the missing values according to their type

In [14]:
# Fill all the numerical speed columns(Kbps speeds + round-trip times) with mean as they were probabli missed 
values_0 = { k:v for (k,v) in zip(numeric_colmns_speed, [raw_df_droped[col].mean() for col in numeric_colmns_speed])} 
raw_df_fill_0 = raw_df_droped.fillna(value=values_0)
percent_missing(raw_df_fill_0)

The dataset contains 11.83 % missing values.


In [15]:
# Fill all the numerical speed ratio columns(%) with 0.0 as the % ratios are probability destributions that need to adduptto 100% 
values_1 = { k:v for (k,v) in zip(numeric_colmns_speed_ratios, [0.0]*len(numeric_colmns_speed_ratios))} 
raw_df_fill_1 = raw_df_fill_0.fillna(value=values_1)
percent_missing(raw_df_fill_1)

The dataset contains 11.75 % missing values.


In [16]:
# Fill the 3 string categorical columns with 'undefined'
values_2 = { k:v for (k,v) in zip(['Handset Type', 'Handset Manufacturer', 'Last Location Name'], ['undefined'] * 3)} 
raw_df_fill_2 = raw_df_fill_1.fillna(value=values_2)
percent_missing(raw_df_fill_2)

The dataset contains 11.72 % missing values.


In [18]:
# Fill the rest of the categorical columns with 0
values_3 = { k:v for (k,v) in zip(['IMEI', 'IMSI', 'MSISDN/Number','bearer id'],[0]*4)}
raw_df_fill_3 = raw_df_fill_2.fillna(value=values_3)
percent_missing(raw_df_fill_3)

The dataset contains 11.69 % missing values.


In [20]:
# Fill the first 3(durations) in  numrical time columns with mean, as they were probably not recorded
values_4 = { k:v for (k,v) in zip(numeric_colmns_time[:3], [raw_df_fill_3[col].mean() for col in numeric_colmns_time[:3]])} 
raw_df_fill_4 = raw_df_fill_3.fillna(value=values_4)
percent_missing(raw_df_fill_4)

The dataset contains 11.69 % missing values.


In [21]:
# Fill the rest of the numrical time columns(No of secs) with 0.0, as that represents the absence of such speed
values_5 = { k:v for (k,v) in zip(numeric_colmns_time[3:], [0.0]*len(numeric_colmns_time[3:]))} 
raw_df_fill_5 = raw_df_fill_4.fillna(value=values_5)
percent_missing(raw_df_fill_5)

The dataset contains 4.22 % missing values.


In [22]:
# Fill the rest of the numrical data volum columns(Bytes) with 0.0, as that represents the absence of data transmission
values_6 = { k:v for (k,v) in zip(numeric_colmns_data_volume, [0.0]*len(numeric_colmns_data_volume))} 
raw_df_fill_6 = raw_df_fill_5.fillna(value=values_6)
percent_missing(raw_df_fill_6)

The dataset contains 0.0 % missing values.


In [23]:
# Fill the first two time-series columns(date-time stamps) formawrd, as they represnt time stamps of events
raw_df_fill_7 = raw_df_fill_6.copy()
raw_df_fill_7.loc[:,time_series_columns[:2]] = raw_df_fill_7.loc[:,time_series_columns[:2]].ffill()
percent_missing(raw_df_fill_7)

The dataset contains 0.0 % missing values.


In [24]:
# Fill the last two time-series columns(milisenc offsets of the time stamps) with their mean
values_8 = { k:v for (k,v) in zip(time_series_columns[2:], [raw_df_fill_7[col].mean() for col in time_series_columns[2:]])} 
raw_df_fill_8 = raw_df_fill_7.fillna(value=values_8)
percent_missing(raw_df_fill_8)

The dataset contains 0.0 % missing values.


#### Convert Bytes to MB,

In [25]:
def convert_bytes_to_megabytes(df, bytes_colmns):
    megabyte = 1048576
    for column in bytes_colmns:
        df[column] = df[column] / megabyte
    return df

df_cleaned = convert_bytes_to_megabytes(raw_df_fill_8, numeric_colmns_data_volume)

In [26]:
#save the final df as pkl dump
df_cleaned['index'] = df_cleaned.index
df_cleaned.to_pickle("../data/df_cleaned.pkl")
colum_descript.to_pickle("../data/colum_descript.pkl")
with open('../data/col_types.pickle', 'wb') as handle:
    pickle.dump({'categorigal':categorical_columns, 'numeric speed':numeric_colmns_speed, 'numeric speed ratio':numeric_colmns_speed_ratios, 
        'numeric time':numeric_colmns_time, 'numeric data':numeric_colmns_data_volume,'time_series':time_series_columns}, handle, protocol=pickle.HIGHEST_PROTOCOL)
del categorical_columns, numeric_colmns_data_volume, numeric_colmns_speed, numeric_colmns_speed_ratios, numeric_colmns_time,time_series_columns,\
    raw_df_fill_8, raw_df_fill_7, raw_df_fill_6, raw_df_fill_5, raw_df_fill_4, raw_df_fill_3,raw_df_fill_2, raw_df_fill_1, raw_df_fill_0,\
        values_0, values_1, values_2, values_3, values_4, values_5, values_6, values_8
gc.collect()

0